<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW2/blob/main/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/.shortcut-targets-by-id/1Ikn75GANDVFyb0klJe4-EBQJikt4_Ob_/NLP


##Extracting essential data



In [3]:
!pip install -q hazm
!pip install -q transformers
!pip install -q dadmatools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.6/862.6 KB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 50.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 KB 22.3 MB/s eta 0:

In [4]:
from __future__ import unicode_literals
import hazm
from hazm import *

from transformers import pipeline, AutoTokenizer


In [5]:
import json
with open('Nlp HW2/extracted-phrases.json','r',encoding='utf-8') as file:
    data = json.load(file)

##Main code

###Config

In [6]:
import itertools

In [7]:
from dadmatools.models.normalizer import Normalizer
import dadmatools.models.lemmatizer
dadma_normalizer = Normalizer(
    full_cleaning=True,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=True,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

dadma_normalizer.normalize("بیا ای عقل کل با من که بردابرد او بینی")

'عقل بردابرد بینی'

In [8]:
import dadmatools.pipeline.language as language

pips = 'lem' 
nlp = language.Pipeline(pips)

# you can see the pipeline with this code
print(nlp.analyze_pipes(pretty=True))

# doc is an SpaCy object
# doc = nlp('از قصهٔ کودکیشان که می‌گفت، گاهی حرص می‌خورد!')


============================= Pipeline Overview =============================

#   Component   Assigns       Requires   Scores   Retokenizes
-   ---------   -----------   --------   ------   -----------
0   tokenizer                                     True       
                                                             
1   lemmatize   token.lemma                       False      

✔ No problems found.
{'summary': {'tokenizer': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': True}, 'lemmatize': {'assigns': ['token.lemma'], 'requires': [], 'scores': [], 'retokenizes': False}}, 'problems': {'tokenizer': [], 'lemmatize': []}, 'attrs': {'token.lemma': {'assigns': ['lemmatize'], 'requires': []}}}


In [9]:
from tqdm import tqdm

In [11]:
corpus = [x[0] for x in tqdm(hazm.words_list())]


100%|██████████| 35717/35717 [00:00<00:00, 1407691.68it/s]


In [ ]:
corpus = [Lemmatizer().lemmatize(word) for word in tqdm(corpus)]

 61%|██████    | 21825/35717 [1:08:54<41:10,  5.62it/s]

In [ ]:
import pickle
with open("corpus.txt","wb") as fle:
  pickle.dump(corpus,fle)

In [94]:
args = {
    'corpus':corpus,
    'pipe':nlp,
    'normalizer':dadma_normalizer,
    'translator':None,
    'word_tokenizer':WordTokenizer(),
    'sent_tokenizer':SentenceTokenizer()
}

In [95]:
class foreign_word_detector():

  def __init__(
      self,
      args
      ):
    self.corpus = args['corpus']
    self.lang_pipe = args['pipe']
    self.normalizer = args['normalizer']
    self.translator = args['translator']
    self.word_tokenizer = args['word_tokenizer']
    self.sent_tokenizer = args['sent_tokenizer']

  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)
    lemmed = [str(self.lang_pipe(sent)) for sent in sentenced]
    normalized = [self.normalizer.normalize(sent) for sent in lemmed]
    words_list = [self.word_tokenizer.tokenize(sent) for sent in normalized]
    words_list = list(itertools.chain(*words_list))

    output = dict()
    out = []
    for word in words_list:
      if word not in self.corpus:
        print(word)
        out.append(word)
        begin = text.find(word)
        output[word] = [begin,begin+len(word)]
    
    return out, output
    

In [96]:
def run():
  detector = foreign_word_detector(args)

  text = ""
  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True : 
    text = input()
    if text == "": 
      break
    
    out, output = detector.detect(text)
    print(out)
    print(output)


In [ ]:
run()

enter your sample text.
enter an empty string if you want to end the process


problem is: using stemmer, we get lots of errors; while using lemmatizer, pronouns are not ommited from the word.


one way to deal with it would be to apply stemming right after dealing with the word origin.

another way would be to find a way to work with the tokenizer introduced in the next cell, since it gives us specific features.

In [ ]:
bolbol_tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
bolbol_tokenizer.tokenize("سیستم کامپیوترم خراب شده.")

['▁سیستم', '▁کامپیوتر', 'م', '▁خراب', '▁شده', '.']